In [1]:
%matplotlib inline


.. currentmodule:: mne_bids


# Convert MEG data to BIDS format

In this notebook you will use MNE-BIDS to organize MEG data according
to the BIDS standard.

In [ ]:
# Notebook is based on tutorial:
# https://mne.tools/mne-bids/stable/auto_examples/convert_mne_sample.html
# Authors: Mainak Jas <mainak.jas@telecom-paristech.fr>
#          Alexandre Gramfort <alexandre.gramfort@telecom-paristech.fr>
#          Teon Brooks <teon.brooks@gmail.com>
#          Stefan Appelhoff <stefan.appelhoff@mailbox.org>
#          Richard Höchenberger <richard.hoechenberger@gmail.com>
#
# License: BSD-3-Clause

In [ ]:
#Install MNE package
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Install MNE BIDS package
pip install mne mne-bids

Note: you may need to restart the kernel to use updated packages.


First we import some basic Python libraries, followed by MNE-Python and its
sample data, and then finally the MNE-BIDS functions we need for this example



In [3]:
import json
import os.path as op
from pprint import pprint
import shutil
import numpy

import mne
from mne.datasets import sample

from mne_bids import (write_raw_bids, read_raw_bids, write_meg_calibration,
                      write_meg_crosstalk, BIDSPath, print_dir_tree,
                      make_dataset_description)
from mne_bids.stats import count_events

Now we can read the MEG data. We define an `event_id` based on our
knowledge of the data, to give meaning to events in the data.

With `raw_fname` and `events`, we determine where to get the sample data
from. `output_path` determines where we will write the BIDS conversion to.



In [4]:
data_path ='/Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG'


raw_fname = op.join(data_path,'tactile_stim_raw_tsss_mc_meg.fif')
#er_fname = op.join(data_path, 'MEG', 'sample', 'ernoise_raw.fif')  # empty room
#events_fname = op.join(data_path, 'MEG', 'sample', 'sample_audvis_raw-eve.fif')


If your data does not conform to MNE naming conventions (all raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz) use following code to load and rename files:

In [7]:
# Load the original raw data file
#raw = mne.io.read_raw(raw_fname)

# Save the raw data to a new file to establish proper filename linkage
#new_fname = '/Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg.fif'
#raw.save(new_fname, overwrite=True)

Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc.fif...
    Range : 126000 ... 1195999 =    126.000 ...  1195.999 secs
Ready.
Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc-1.fif...
    Range : 1196000 ... 2265999 =   1196.000 ...  2265.999 secs
Ready.
Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc-2.fif...
    Range : 2266000 ... 3118999 =   2266.000 ...  3118.999 secs
Ready.


/var/folders/8j/2w7rdjlj5sg59_1hy83zsg3c0000gn/T/ipykernel_52924/937860091.py:2: RuntimeWarning: This filename (/Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(raw_fname)


Writing /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg.fif
Overwriting existing file.
Writing /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg-1.fif
Overwriting existing file.
Writing /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg-2.fif
Closing /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg-2.fif
Closing /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg-1.fif
Closing /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg.fif
[done]


In [5]:
output_path = op.join(data_path, 'BIDS')

To ensure the output path doesn't contain any leftover files from previous
tests and example runs, we simply delete it.

<div class="alert alert-danger"><h4>Warning</h4><p>Do not delete directories that may contain important data!</p></div>




In [6]:
if op.exists(output_path):
    shutil.rmtree(output_path)

<div class="alert alert-info"><h4>Note</h4><p>``mne-bids`` will try to infer as much information from the data as
  possible to then save this data in BIDS-specific "sidecar" files. For
  example the manufacturer information, which is inferred from the data file
  extension. However, sometimes inferring is ambiguous (e.g., if your file
  format is non-standard for the manufacturer). In these cases, MNE-BIDS does
  *not* guess and you will have to update your BIDS fields manually.</p></div>

Based on our path definitions above, we read the raw data file, define
a new BIDS name for it, and then run the automatic BIDS conversion for both
the experimental data and its associated empty-room recording.



In [ ]:
raw = mne.io.read_raw(raw_fname)

# specify power line frequency as required by BIDS
raw.info['line_freq'] = 60


task = 'facerecognition'
bids_path = BIDSPath(
    subject='01',
    session='01',
    task='face',
    run='1',
    root=output_path
)
write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    overwrite=True
)

Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg.fif...
    Range : 126000 ... 1249999 =    126.000 ...  1249.999 secs
Ready.
Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg-1.fif...
    Range : 1250000 ... 2373999 =   1250.000 ...  2373.999 secs
Ready.
Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg-2.fif...
    Range : 2374000 ... 3118999 =   2374.000 ...  3118.999 secs
Ready.
Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg.fif...
    Range : 126000 ... 1249999 =    126.000 ...  1249.999 secs
Ready.
Opening raw data file /Users/yngvifrey/Documents/Data_MEEG/MEEG_data/NatMEG_0177/170424/MEG/tactile_stim_raw_tsss_mc_meg-1.fif...
    Range : 1250000 ... 2373999 =   1250.000 ...  2373.999 secs
Ready.
O

/var/folders/8j/2w7rdjlj5sg59_1hy83zsg3c0000gn/T/ipykernel_54149/3953831482.py:15: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events and event_id parameters. For resting state data, BIDS recommends naming the task using labels beginning with "rest".
  write_raw_bids(


Let's pause and check that the information that we've written out to the
sidecar files that describe our data is correct.



In [ ]:
# Get the sidecar ``.json`` file
sidecar_json_bids_path = bids_path.copy().update(extension='.json')
sidecar_json_content = sidecar_json_bids_path.fpath.read_text(
    encoding='utf-8-sig'
)
print(sidecar_json_content)

The sample MEG dataset comes with fine-calibration and crosstalk files that
are required when processing Elekta/Neuromag/MEGIN data using MaxFilter®.
Let's store these data in appropriate places, too.



In [ ]:
cal_fname = op.join(data_path, 'SSS', 'sss_cal_mgh.dat')
ct_fname = op.join(data_path, 'SSS', 'ct_sparse_mgh.fif')

write_meg_calibration(cal_fname, bids_path)
write_meg_crosstalk(ct_fname, bids_path)

Now let's see the structure of the BIDS folder we created.



In [ ]:
print_dir_tree(output_path)

Now let's get an overview of the events on the whole dataset



In [ ]:
counts = count_events(output_path)
counts

A big advantage of having data organized according to BIDS is that software
packages can automate your workflow. For example, reading the data back
into MNE-Python can easily be done using :func:`read_raw_bids`.



In [ ]:
raw = read_raw_bids(bids_path=bids_path)

The resulting data is already in a convenient form to create epochs and
evoked data.



In [ ]:
events, event_id = mne.events_from_annotations(raw)
epochs = mne.Epochs(raw, events, event_id)
epochs['Auditory'].average().plot()

We can easily get the :class:`mne_bids.BIDSPath` of the empty-room recording
that was associated with the experimental data while writing. The empty-room
data can then be loaded with :func:`read_raw_bids`.



In [ ]:
er_bids_path = bids_path.find_empty_room(use_sidecar_only=True)
er_data = read_raw_bids(er_bids_path)
er_data

It is trivial to retrieve the path of the fine-calibration and crosstalk
files, too.



In [ ]:
print(bids_path.meg_calibration_fpath)
print(bids_path.meg_crosstalk_fpath)

The README created by :func:`write_raw_bids` also takes care of the citation
for mne-bids. If you are preparing a manuscript, please make sure to also
cite MNE-BIDS there.



In [ ]:
readme = op.join(output_path, 'README')
with open(readme, 'r', encoding='utf-8-sig') as fid:
    text = fid.read()
print(text)

It is also generally a good idea to add a description of your dataset,
see the `BIDS dataset_description.json definition`_ for more information.



In [ ]:
how_to_acknowledge = """\
If you reference this dataset in a publication, please acknowledge its \
authors and cite MNE papers: A. Gramfort, M. Luessi, E. Larson, D. Engemann, \
D. Strohmeier, C. Brodbeck, L. Parkkonen, M. Hämäläinen, \
MNE software for processing MEG and EEG data, NeuroImage, Volume 86, \
1 February 2014, Pages 446-460, ISSN 1053-8119 \
and \
A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck, \
R. Goj, M. Jas, T. Brooks, L. Parkkonen, M. Hämäläinen, MEG and EEG data \
analysis with MNE-Python, Frontiers in Neuroscience, Volume 7, 2013, \
ISSN 1662-453X"""

make_dataset_description(
    path=bids_path.root,
    name=task,
    authors=["Alexandre Gramfort", "Matti Hämäläinen"],
    how_to_acknowledge=how_to_acknowledge,
    acknowledgements="""\
Alexandre Gramfort, Mainak Jas, and Stefan Appelhoff prepared and updated the \
data in BIDS format.""",
    data_license='CC0',
    ethics_approvals=['Human Subjects Division at the University of Washington'],  # noqa: E501
    funding=[
        "NIH 5R01EB009048",
        "NIH 1R01EB009048",
        "NIH R01EB006385",
        "NIH 1R01HD40712",
        "NIH 1R01NS44319",
        "NIH 2R01NS37462",
        "NIH P41EB015896",
        "ANR-11-IDEX-0003-02",
        "ERC-StG-263584",
        "ERC-StG-676943",
        "ANR-14-NEUC-0002-01"
    ],
    references_and_links=[
        "https://doi.org/10.1016/j.neuroimage.2014.02.017",
        "https://doi.org/10.3389/fnins.2013.00267",
        "https://mne.tools/stable/overview/datasets_index.html#sample"
    ],
    doi="doi:10.18112/openneuro.ds000248.v1.2.4",
    overwrite=True
)
desc_json_path = bids_path.root / 'dataset_description.json'
with open(desc_json_path, 'r', encoding='utf-8-sig') as fid:
    pprint(json.loads(fid.read()))

This should be very similar to the `ds000248 dataset_description.json`_!

